<a href="https://colab.research.google.com/github/Matasnet/Interaktywny-notatnik/blob/main/Projekt_interaktywna_wycena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalacja bibliotek lub ich aktualizacja  


In [25]:
#!pip install pandas matplotlib seaborn ipywidgets


#Import potrzebnych bibliotek

In [26]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from google.colab import files  # Jeśli używasz Google Colab

## Inicjalizacja pustych tabeli

In [27]:
wycena_data = {
    'Nazwa': [],
    'Jednostka miary': [],
    'Ilość': [],
    'Cena jednostkowa (zł)': [],
    'Koszt (zł)': [],
    'Marża (%)': [],
    'Rabat (%)': []
}

## Tworzenie DataFrame

In [28]:
df_wycena = pd.DataFrame(wycena_data)

## Widgety do wprowadzania danych


In [29]:
# Widgety do wprowadzania danych
nazwa_input = widgets.Text(placeholder='Nazwa materiału')
jednostka_input = widgets.Dropdown(
    options=['kg', 'm', 'szt', 'l', 'm²', 'm³', 'op'],  # Możesz dodać więcej jednostek
    description='Jednostka:'
)
ilosc_input = widgets.IntText(value=1, description='Ilość:')
cena_input = widgets.FloatText(value=0.0, description='Cena za jednostkę (zł):')
marza_input = widgets.FloatText(value=20.0, description='Marża na produkt (%):')  # Domyślna marża
rabat_input = widgets.FloatText(value=0.0, description='Rabat na produkt (%):')  # Rabat
usun_input = widgets.IntText(value=0, description='Nr wiersza do usunięcia:')  # Numer wiersza do usunięcia

# Output widget do wyświetlania tabeli
output = widgets.Output()


##Funkcje interaktywnej wyceny

In [30]:
# Funkcja do odświeżania tabeli w jednym miejscu
def wyswietl_tabele():
    global df_wycena
    with output:
        output.clear_output()  # Czyszczenie poprzedniego wyniku
        display(df_wycena)

# Funkcja do dodawania nowych wierszy do tabeli
def dodaj_pozycje(b):
    global df_wycena
    nazwa = nazwa_input.value
    jednostka = jednostka_input.value
    ilosc = ilosc_input.value
    cena_jednostkowa = cena_input.value
    koszt = ilosc * cena_jednostkowa
    marza = marza_input.value  # Marża dla tego produktu
    rabat = rabat_input.value  # Rabat dla tego produktu

    # Nowa pozycja jako DataFrame
    nowa_pozycja = pd.DataFrame({
        'Nazwa': [nazwa],
        'Jednostka miary': [jednostka],
        'Ilość': [ilosc],
        'Cena jednostkowa (zł)': [cena_jednostkowa],
        'Koszt (zł)': [koszt],
        'Marża (%)': [marza],
        'Rabat (%)': [rabat]
    })

    # Dodanie nowej pozycji do istniejącej tabeli przy użyciu concat
    df_wycena = pd.concat([df_wycena, nowa_pozycja], ignore_index=True)

    # Odświeżenie tabeli
    wyswietl_tabele()

# Funkcja do usuwania wiersza na podstawie indeksu
def usun_pozycje(b):
    global df_wycena
    nr_wiersza = usun_input.value

    # Sprawdzamy, czy numer wiersza jest prawidłowy
    if 0 <= nr_wiersza < len(df_wycena):
        df_wycena = df_wycena.drop(index=nr_wiersza).reset_index(drop=True)
        # Odświeżenie tabeli
        wyswietl_tabele()
    else:
        print(f"Wiersz o numerze {nr_wiersza} nie istnieje.")

# Funkcja do obliczenia kosztu z marżą i rabatem dla każdego produktu
def oblicz_marze_i_rabat(b):
    global df_wycena

    # Przeliczenie marży i rabatu dla każdego wiersza
    df_wycena['Koszt z marżą (zł)'] = df_wycena['Koszt (zł)'] * (1 + df_wycena['Marża (%)'] / 100)
    df_wycena['Koszt po rabacie (zł)'] = df_wycena['Koszt z marżą (zł)'] * (1 - df_wycena['Rabat (%)'] / 100)

    # Odświeżenie tabeli
    wyswietl_tabele()

# Funkcja do eksportu do CSV
def eksportuj_csv(b):
    global df_wycena
    df_wycena.to_csv('wycena.csv', index=False)
    files.download('wycena.csv')

# Funkcja do eksportu do Excel
def eksportuj_excel(b):
    global df_wycena
    df_wycena.to_excel('wycena.xlsx', index=False)
    files.download('wycena.xlsx')

##Dodanie przycisków

In [31]:
# Przycisk dodawania pozycji
dodaj_button = widgets.Button(description="Dodaj pozycję")
dodaj_button.on_click(dodaj_pozycje)

# Przycisk do naliczenia marży i rabatu
marza_rabat_button = widgets.Button(description="Oblicz marżę i rabat")
marza_rabat_button.on_click(oblicz_marze_i_rabat)

# Przycisk do usunięcia pozycji
usun_button = widgets.Button(description="Usuń pozycję")
usun_button.on_click(usun_pozycje)

# Przycisk do eksportu do CSV
eksportuj_csv_button = widgets.Button(description="Eksportuj do CSV")
eksportuj_csv_button.on_click(eksportuj_csv)

# Przycisk do eksportu do Excel
eksportuj_excel_button = widgets.Button(description="Eksportuj do Excel")
eksportuj_excel_button.on_click(eksportuj_excel)

# Wyświetlenie widgetów i przycisków
display(nazwa_input, jednostka_input, ilosc_input, cena_input, marza_input, rabat_input, dodaj_button)
display(marza_rabat_button)
display(usun_input, usun_button)
display(eksportuj_csv_button, eksportuj_excel_button)

# Wyświetlenie tabeli (pustej na początku)
display(output)

Text(value='', placeholder='Nazwa materiału')

Dropdown(description='Jednostka:', options=('kg', 'm', 'szt', 'l', 'm²', 'm³', 'op'), value='kg')

IntText(value=1, description='Ilość:')

FloatText(value=0.0, description='Cena za jednostkę (zł):')

FloatText(value=20.0, description='Marża na produkt (%):')

...

IntText(value=0, description='Nr wiersza do usunięcia:')

Button(description='Usuń pozycję', style=ButtonStyle())

Button(description='Eksportuj do CSV', style=ButtonStyle())

Button(description='Eksportuj do Excel', style=ButtonStyle())

Output()

W celu zrestartowania danych należy ponownie uruchomić ostatnią komórkę.